In [39]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import requests
import csv
%matplotlib inline
import seaborn as sns
%pylab inline
pd.options.display.mpl_style = 'default'
import matplotlib
matplotlib.style.use('ggplot')
pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns',40)

Populating the interactive namespace from numpy and matplotlib


In [40]:
# Read file
loanData = pd.read_csv('/Users/chrispiccin/nyc16_ids1/data/Lending_Club/LoanStats3a.csv')    

In [41]:
loanData.shape

(42536, 111)

In [42]:
# Read and concatenate remaining files
loanData2 = pd.read_csv('/Users/chrispiccin/nyc16_ids1/data/Lending_Club/LoanStats3b.csv')

In [43]:
loanData2.shape

(188123, 111)

In [44]:
loanData3 = pd.read_csv('/Users/chrispiccin/nyc16_ids1/data/Lending_Club/LoanStats3c.csv')    # Supply the file name (path)

In [45]:
loanData3.shape

(235629, 111)

In [46]:
loanData4 = pd.read_csv('/Users/chrispiccin/nyc16_ids1/data/Lending_Club/LoanStats3d.csv')    # Supply the file name (path)

In [47]:
loanData4.shape

(421095, 111)

In [48]:
LoanData = pd.concat([loanData, loanData2, loanData3, loanData4], ignore_index=True)

In [49]:
# Test dropping columns by name
LoanData.drop(LoanData[['funded_amnt','funded_amnt_inv']], axis=1, inplace=True)

In [50]:
LoanData.shape

(887383, 109)

In [51]:
# Drop remaining columns
LoanData.drop(LoanData[['installment','emp_title','pymnt_plan','desc','title','initial_list_status','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt','next_pymnt_d','last_credit_pull_d','collections_12_mths_ex_med','policy_code','annual_inc_joint','dti_joint','verification_status_joint','open_acc_6m','open_il_6m','open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m','acc_open_past_24mths','avg_cur_bal','bc_open_to_buy','bc_util','chargeoff_within_12_mths','delinq_amnt','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mort_acc','mths_since_recent_bc','mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m','pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies','tax_liens','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit']], axis=1, inplace=True)

In [52]:
LoanData.shape

(887383, 33)

In [72]:
LoanData.head()

,id,member_id,loan_amnt,term,int_rate,emp_length,annual_inc,verification_status,issue_d,loan_status,url,zip_code,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,...,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,yrs_since_first_cr_line
0,1077501,1296599.0,5000.0,0,10.65,0,24000.0,0,Dec-2011,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,860xx,27.65,0.0,Jan-1985,1.0,-1.0,-1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1077430,1314167.0,2500.0,1,15.27,1,30000.0,1,Dec-2011,Charged Off,https://lendingclub.com/browse/loanDetail.acti...,309xx,1.00,0.0,Apr-1999,5.0,-1.0,-1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1077175,1313524.0,2400.0,0,15.96,0,12252.0,0,Dec-2011,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,606xx,8.72,0.0,Nov-2001,2.0,-1.0,-1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1076863,1277178.0,10000.0,0,13.49,0,49200.0,1,Dec-2011,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,917xx,20.00,0.0,Feb-1996,1.0,35.0,-1.0,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1075269,1311441.0,5000.0,0,7.90,3,36000.0,1,Dec-2011,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,852xx,11.20,0.0,Nov-2004,3.0,-1.0,-1.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
# Convert 'term' to catigorical
LoanData['term'] = [0 if x == " 36 months" else 1 for x in LoanData['term']]

In [71]:
# Clean up interest rate
LoanData['int_rate'] = LoanData['int_rate'].str.replace('%', '')
LoanData['int_rate'] = LoanData['int_rate'].astype(float)

In [54]:
# Cleaning up 'emp_length'
emp_years = dict(zip(LoanData['emp_length'].unique(), np.arange(LoanData['emp_length'].nunique())))
LoanData['emp_length'] = LoanData['emp_length'].map(lambda x: emp_years[x])

In [ ]:
# Convert 'verification_status' to categorical
LoanData['verification_status'] = [1 if x == "Source Verified" else 0 for x in LoanData['verification_status']]

In [73]:
# clean up issue_month and issue_year
LoanData['issue_month'] = 0
LoanData['issue_year'] = 0
LoanData['issue_month'] = [pd.to_datetime(x).month for x in LoanData['issue_d']]
LoanData['issue_year'] = [pd.to_datetime(x).year for x in LoanData['issue_d']]
LoanData = LoanData.drop('issue_d', 1)

In [74]:
# Convert 'loan_status' to categorical
LoanData['loan_status'] = [0 if x in ['Charged Off','Default','Does not meet the credit policy.  Status:Charged Off','Does not meet the credit policy.  Status:Default'] else 1 for x in loanData['loan_status']]

ValueError: Length of values does not match length of index

In [ ]:
# Clean up zip_code
LoanData['zip_code'] = [x[:3] for x in LoanData['zip_code']]

In [ ]:
# Drop nulls in 'delinq_2yrs'
LoanData['delinq_2yrs'] = [-1 if pd.isnull(x) else x for x in LoanData['delinq_2yrs']]

In [75]:
# Clean up 'yrs_since_first_cr_line'
LoanData['first_cr_line'] = 0
LoanData['first_cr_line'] = [date.today().year-pd.to_datetime(x).year for x in LoanData['earliest_cr_line']]
LoanData = LoanData.drop('earliest_cr_line', 1)

NameError: name 'date' is not defined

In [55]:
# Convert 'grade' to categorical
LoanData = LoanData[pd.isnull(LoanData['grade']) == 0]
LoanData.index = range(len(LoanData))
grades_cat = pd.get_dummies(LoanData['grade'])
LoanData = pd.concat([LoanData, grades_cat], axis=1)
LoanData = LoanData.drop(['grade'], 1)

In [56]:
# Convert 'sub_grade' to categorical
LoanData = LoanData[pd.isnull(LoanData['sub_grade']) == 0]
LoanData.index = range(len(LoanData))
grades_cat = pd.get_dummies(LoanData['sub_grade'])
LoanData = pd.concat([LoanData, grades_cat], axis=1)
LoanData = LoanData.drop(['sub_grade'], 1)

In [58]:
# Convert 'home_ownership' to categorical
home_own = pd.get_dummies(LoanData['home_ownership'])
LoanData = pd.concat([LoanData, home_own], axis=1)
LoanData = LoanData.drop(['home_ownership'], 1)

In [60]:
# Convert 'purpose' to categorical
purposes = pd.get_dummies(LoanData['purpose'])
LoanData = pd.concat([LoanData, purposes], axis=1)
LoanData = LoanData.drop('purpose', 1)

In [61]:
# Drop nulls in 'mths_since_last_delinq'
LoanData['mths_since_last_delinq'] = [-1 if pd.isnull(x) else x for x in LoanData['mths_since_last_delinq']]

In [62]:
# Drop nulls in 'mths_since_last_record'
LoanData['mths_since_last_record'] = [-1 if pd.isnull(x) else x for x in LoanData['mths_since_last_record']]

In [64]:
# Clean up 'revol_util'
LoanData['revol_util'] = LoanData['revol_util'].str.replace('%', '')
LoanData['revol_util'] = LoanData['revol_util'].astype(float)
LoanData['revol_util'] = [-1 if pd.isnull(x) else x for x in LoanData['revol_util']]

In [65]:
# Drop nulls from 'mths_since_last_major_derog'
LoanData['mths_since_last_major_derog'] = [-1 if pd.isnull(x) else x for x in LoanData['mths_since_last_major_derog']]

In [66]:
# Removing loans that are still in progress
LoanData = LoanData[[x in ['Charged Off','Default','Fully Paid','Does not meet the credit policy.  Status:Charged Off','Does not meet the credit policy.  Status:Default','Does not meet the credit policy.  Status:Fully Paid'] for x in LoanData['loan_status']]]
LoanData.index = range(len(LoanData))

In [30]:
# Relabel all defaulted loans as charged off and number them
LoanData['loan_status'] = [0 if x in ['Charged Off','Default','Does not meet the credit policy.  Status:Charged Off','Does not meet the credit policy.  Status:Default'] else 1 for x in loanData['loan_status'] ]

ValueError: Length of values does not match length of index

In [69]:
# Convert 'addr_state' to categorical
states = pd.get_dummies(LoanData['addr_state'])
LoanData = pd.concat([LoanData, states], axis=1)
LoanData = LoanData.drop('addr_state', 1)

KeyError: 'addr_state'

NameError: name 'date' is not defined

In [37]:
LoanData.head(3)

,id,member_id,loan_amnt,term,int_rate,emp_length,annual_inc,verification_status,issue_d,loan_status,url,zip_code,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,...,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,yrs_since_first_cr_line,issue_month,issue_year
0,1077501,1296599.0,5000.0,0,10.65,0,24000.0,0,Dec-2011,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,860,27.65,0.0,Jan-1985,1.0,-1.0,-1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,12,2011
1,1077430,1314167.0,2500.0,1,15.27,1,30000.0,1,Dec-2011,Charged Off,https://lendingclub.com/browse/loanDetail.acti...,309,1.00,0.0,Apr-1999,5.0,-1.0,-1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,12,2011
2,1077175,1313524.0,2400.0,0,15.96,0,12252.0,0,Dec-2011,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,606,8.72,0.0,Nov-2001,2.0,-1.0,-1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,12,2011


In [49]:
LoanData.dtypes

id                              object
member_id                       object
loan_amnt                      float64
term                            object
int_rate                       float64
grade                           object
sub_grade                       object
emp_length                      object
home_ownership                  object
annual_inc                     float64
verification_status             object
issue_d                         object
loan_status                     object
url                             object
purpose                         object
zip_code                        object
addr_state                      object
dti                            float64
delinq_2yrs                    float64
earliest_cr_line                object
inq_last_6mths                 float64
mths_since_last_delinq         float64
mths_since_last_record         float64
open_acc                       float64
pub_rec                        float64
revol_bal                

In [37]:
LoanData.loan_status.value_counts()

Current                                                555383
Fully Paid                                             251122
Charged Off                                             56773
Late (31-120 days)                                      11763
In Grace Period                                          7146
Late (16-30 days)                                        2165
Does not meet the credit policy. Status:Fully Paid       1988
Does not meet the credit policy. Status:Charged Off       761
Default                                                   281
Name: loan_status, dtype: int64

In [44]:
LoanData.earliest_cr_line.value_counts()

Aug-2001    6659
Aug-2000    6529
Oct-2000    6322
Oct-2001    6154
Aug-2002    6086
Sep-2000    5918
Nov-1999    5878
Oct-1999    5824
Nov-2000    5822
Sep-2002    5812
Oct-2002    5684
Sep-2001    5678
Sep-2003    5612
Aug-1999    5598
Sep-1999    5588
Jul-2001    5330
Nov-2001    5223
Oct-2003    5212
Aug-2003    5205
Mar-2001    5173
Sep-1998    5141
Dec-2000    5103
Sep-2004    5075
May-2001    5067
Dec-2001    5046
Dec-1999    5019
Jul-2000    4997
Oct-1998    4946
Jun-2001    4930
Mar-2000    4906
Jul-2002    4863
May-2000    4829
Jun-2000    4817
Jan-2001    4774
Jul-2003    4742
Apr-2001    4714
Dec-1998    4712
Nov-1998    4710
Aug-2004    4655
Aug-1998    4654
Feb-2000    4613
May-2002    4567
Nov-2002    4541
Jul-1999    4491
Oct-2004    4487
Jan-2000    4469
Mar-1999    4460
Jan-2002    4430
Mar-2002    4417
Oct-1997    4413
May-2003    4400
Feb-2001    4368
Apr-2002    4354
Nov-2003    4332
Apr-2000    4290
Feb-2002    4261
Dec-2002    4257
Jun-1999    4159
Jun-2002    41

In [38]:
#pd.to_datetime(LoanData['issue_d'])

In [42]:
LoanData.dtypes

dtype('<M8[ns]')

In [38]:
LoanData.issue_d.value_counts()

Oct-2015    48631
Jul-2015    45962
Dec-2015    44343
Oct-2014    38783
Nov-2015    37530
Aug-2015    35886
Apr-2015    35427
Jan-2015    35107
May-2015    31913
Jul-2014    29306
Sep-2015    28641
Jun-2015    28485
Mar-2015    25400
Nov-2014    25054
Feb-2015    23770
May-2014    19099
Apr-2014    19071
Aug-2014    18814
Jun-2014    17179
Mar-2014    16513
Jan-2014    15628
Feb-2014    15269
Dec-2013    15020
Nov-2013    14676
Oct-2013    14115
Sep-2013    12987
Aug-2013    12674
Jul-2013    11910
Jun-2013    10899
Sep-2014    10606
May-2013    10350
Dec-2014    10307
Apr-2013     9419
Mar-2013     8273
Feb-2013     7561
Jan-2013     6872
Nov-2012     6382
Oct-2012     6263
Sep-2012     6087
Dec-2012     6066
Aug-2012     5419
Jul-2012     4627
Jun-2012     3817
May-2012     3400
Apr-2012     3230
Mar-2012     2914
Jan-2012     2602
Feb-2012     2560
Dec-2011     2267
Nov-2011     2232
Oct-2011     2118
Sep-2011     2067
Aug-2011     1934
Jul-2011     1875
Jun-2011     1835
May-2011  